# Chose best k value for KNN imputer

In [141]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sns.set()

from sklearn.impute import KNNImputer

In [142]:
df = pd.read_csv("ristorazione_5096_id_3_anni.csv")
df = df[df.columns[7:-29]] # i select only input columns for my model and i also excluded categorical columns
df["Anno di costituzione"] = pd.to_datetime(df["Anno di costituzione"])
df["Anno di costituzione"] = df["Anno di costituzione"].apply(lambda x : 2019 - x.year)
df

,Anno di costituzione,Dipendenti\nAnno - 1,Dipendenti\nAnno - 2,Dipendenti\nAnno - 3,Ricavi vendite e prestazioni\nmigl EUR\nAnno - 1,Ricavi vendite e prestazioni\nmigl EUR\nAnno - 2,Ricavi vendite e prestazioni\nmigl EUR\nAnno - 3,TOT. VAL. DELLA PRODUZIONE\nmigl EUR\nAnno - 1,TOT. VAL. DELLA PRODUZIONE\nmigl EUR\nAnno - 2,TOT. VAL. DELLA PRODUZIONE\nmigl EUR\nAnno - 3,...,Redditività di tutto il capitale investito (ROI) (%)\n%\nAnno - 3,Rotaz. cap. investito (volte)\nAnno - 1,Rotaz. cap. investito (volte)\nAnno - 2,Rotaz. cap. investito (volte)\nAnno - 3,Indice corrente\nAnno - 1,Indice corrente\nAnno - 2,Indice corrente\nAnno - 3,Rapporto di indebitamento\nAnno - 1,Rapporto di indebitamento\nAnno - 2,Rapporto di indebitamento\nAnno - 3
0,74,12318,12339,12397,5.685940e+08,5.666451e+08,5.679715e+08,5.791711e+08,575005635.0,5.816119e+08,...,0.98,1.03,0.98,0.98,0.92,0.95,0.92,2.63,2.76,2.86
1,42,11663,11744,11806,6.036289e+08,5.798254e+08,5.593294e+05,6.231545e+05,595220388.0,5.715666e+08,...,8.12,1.36,1.33,1.37,1.36,1.35,1.33,2.46,2.58,2.69
2,48,10385,10466,10275,4.198258e+08,4.212691e+08,4.036268e+08,4.360835e+08,438296558.0,4.227815e+08,...,25.24,1.93,1.9,1.96,1.67,1.61,1.72,2.91,3.09,2.86
3,37,7009,6592,6600,5.942300e+05,5.472078e+08,5.070680e+05,5.992040e+08,551965897.0,5.132697e+05,...,11.94,1.93,1.78,1.57,1.18,1.11,1.01,2.39,2.59,3
4,13,139,113,61,9.197681e+06,7.713829e+06,4.674376e+06,9.295149e+06,7981088.0,4.959085e+06,...,n.s.,1.96,1.88,1.5,1.01,0.89,0.83,2.79,4.17,5.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5091,13,6,5,5,3.905280e+05,4.178080e+05,4.306910e+05,4.313380e+05,417917.0,4.309220e+05,...,19.95,1.19,1.59,1.5,2.11,2.38,1.89,6.71,4.33,5.48
5092,17,11,10,9,8.768720e+05,7.555050e+05,6.971030e+05,8.786660e+05,758363.0,6.973620e+05,...,n.s.,4.72,4.08,4.27,2.85,3.55,3.46,2.28,1.80,1.79
5093,32,6,7,6,6.394590e+05,6.596000e+02,6.928710e+05,6.416650e+05,666044.0,6.970520e+05,...,7.68,0.75,0.78,0.8,1.77,1.1,1.3,4.94,6.32,8.93
5094,26,9,5,5,3.900880e+05,3.928600e+02,4.065390e+05,3.914780e+05,394866.0,4.202590e+05,...,n.d.,2.42,2.49,2.45,7.1,5.78,2.98,1.54,1.52,1.65


### Replace n.s. and n.s. all of which mean null

In [143]:
df.replace("n.s.",float("nan"),inplace=True)
df.replace("n.d.",float("nan"),inplace=True)
for col in df.columns:
    df[col] = df[col].apply(lambda x: float(x))

### Total number of null values

In [144]:
df.isna().sum().sum()

10181

### Total number of null values by column

In [145]:
df.isna().sum()

Anno di costituzione                                 0
Dipendenti\nAnno - 1                                 3
Dipendenti\nAnno - 2                                 4
Dipendenti\nAnno - 3                                10
Ricavi vendite e prestazioni\nmigl EUR\nAnno - 1     0
                                                    ..
Indice corrente\nAnno - 2                           47
Indice corrente\nAnno - 3                           56
Rapporto di indebitamento\nAnno - 1                  0
Rapporto di indebitamento\nAnno - 2                  0
Rapporto di indebitamento\nAnno - 3                  2
Length: 76, dtype: int64

## Creating a dataset with an extra 1% of dummy missing data

In [146]:
df_fake_na = df.copy()
for col in df_fake_na.columns:
    ori_rat = df_fake_na[col].isna().mean()

    add_miss_rat = 0.01 * (1+2*(df_fake_na[col].isna().sum()/5096)) # balance fake null between columns with more or less null
    vals_to_nan = df_fake_na[col].dropna().sample(frac=add_miss_rat).index
    df_fake_na.loc[vals_to_nan, col] = float("nan")


In [147]:
df_fake_na.isna().sum().sum()

14069

In [148]:
df_fake_na.isna().sum()

Anno di costituzione                                 51
Dipendenti\nAnno - 1                                 54
Dipendenti\nAnno - 2                                 55
Dipendenti\nAnno - 3                                 61
Ricavi vendite e prestazioni\nmigl EUR\nAnno - 1     51
                                                   ... 
Indice corrente\nAnno - 2                            98
Indice corrente\nAnno - 3                           108
Rapporto di indebitamento\nAnno - 1                  51
Rapporto di indebitamento\nAnno - 2                  51
Rapporto di indebitamento\nAnno - 3                  53
Length: 76, dtype: int64

# Test filling error using mean squared error

In [149]:
from sklearn.metrics import mean_squared_error

### First have another look of true nan by columns

In [150]:
px.bar(df.isna().sum(),text_auto=True, title="True Null values by columns")

In [151]:
px.bar(100*(df[df.columns[-15:-3]].isna().sum())/5096,text_auto=".3", title="True Null values % by columns (zoom)")

In [152]:
px.bar(100*(df_fake_na[df_fake_na.columns[-15:-3]].isna().sum())/5096,text_auto=".3", title="True + Fake Null values % by columns (zoom)")

### Keep in mind that our imputer must perform better on these columns, so we should weight errors on those columns heavier (see next cell)

---

### Test for k between 1 and 10

In [153]:
df_evaluation = pd.DataFrame(columns=["k = " + str(k) for k in range(1,16)], index = df.columns)
df_errors = pd.DataFrame(columns=["k = " + str(k) for k in range(1,16)],index=["MSE","MSE principal nan columns"])
for k in range(1,16):
    imputer = KNNImputer(n_neighbors=k) #create imputer
    df_full_filled = pd.DataFrame(data = imputer.fit_transform(df_fake_na), columns=df.columns, index=df.index) #use on dataset with fake nan
    df_result = (100*(df_full_filled - df)/ df) # percentage error #The null of the original dataset will propagate back to df_result 
    df_result.replace(float("nan"), value=0,inplace=True) #get rid of the true null, make them zero (they will not count in error calculation)
    df_result.replace(float("inf"), value=0,inplace=True) #get rid of some inf value causated by a few zeros in the original dataset
    df_zeros = df_result * 0  # df_zeros will be our ideal perfect result (all diff between df_filled and df to zero)
    mse = mean_squared_error(df_zeros, df_result/100) #calculating mse on all the dataset 
    mse_nans_cols = mean_squared_error(df_zeros[df.columns[-15:-3]], (df_result[df.columns[-15:-3]])/100) #calculating mse on all the dataset on most important columns
    print("-----------------") #print the result and save on df_evaluation and df_errors
    print("Results for k = "+str(k))
    print("Mse: "+str(mse))
    print("Mse principal nan columns: "+str(mse_nans_cols))
    df_evaluation["k = " + str(k)] = list(df_result.mean())
    df_errors["k = " + str(k)] = [mse,mse_nans_cols]
    

-----------------
Results for k = 1
Mse: 160378.41691635567
Mse principal nan columns: 182.17003502218668
-----------------
Results for k = 2
Mse: 155026.94516537216
Mse principal nan columns: 40.84006114304241
-----------------
Results for k = 3
Mse: 163658.790773146
Mse principal nan columns: 21.804503079344745
-----------------
Results for k = 4
Mse: 245606.09124601123
Mse principal nan columns: 14.035730893629959
-----------------
Results for k = 5
Mse: 324940.7394162503
Mse principal nan columns: 14.532439698388812
-----------------
Results for k = 6
Mse: 491435.6568254553
Mse principal nan columns: 9.149938720406029
-----------------
Results for k = 7
Mse: 692037.9853661237
Mse principal nan columns: 8.202293711592054
-----------------
Results for k = 8
Mse: 596214.9261507913
Mse principal nan columns: 8.609428177067619
-----------------
Results for k = 9
Mse: 548814.6660798108
Mse principal nan columns: 7.093426269853906
-----------------
Results for k = 10
Mse: 498597.318855255

## Visualize results

In [154]:
df_evaluation

,k = 1,k = 2,k = 3,k = 4,k = 5,k = 6,k = 7,k = 8,k = 9,k = 10,k = 11,k = 12,k = 13,k = 14,k = 15
Anno di costituzione,0.066910,0.149397,0.155386,0.170030,0.169148,0.205028,0.205119,0.195907,0.196206,0.205095,0.208618,0.206702,0.203886,0.198233,0.202622
Dipendenti\nAnno - 1,0.073307,0.144106,0.130057,0.077344,0.081644,0.115844,0.110623,0.114543,0.108149,0.100151,0.095845,0.101727,0.112542,0.102785,0.105196
Dipendenti\nAnno - 2,0.196227,0.243185,0.222575,0.207877,0.210815,0.202249,0.227317,0.236690,0.287321,0.276153,0.278423,0.278510,0.263452,0.262321,0.248906
Dipendenti\nAnno - 3,0.220118,0.104986,0.224179,0.270552,0.265478,2.836749,2.458232,2.192755,3.490846,3.194441,3.805734,3.532926,3.300437,3.109879,2.935187
Ricavi vendite e prestazioni\nmigl EUR\nAnno - 1,69.723456,54.504137,41.573753,50.926805,50.976469,50.687619,51.855783,51.540169,51.921776,50.462014,51.608786,52.484516,52.486768,53.072523,53.217032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Indice corrente\nAnno - 2,0.428691,0.351834,0.400156,0.396842,0.430051,0.415687,0.422864,0.474035,0.475194,0.510123,0.502452,0.506896,0.522955,0.589745,0.594619
Indice corrente\nAnno - 3,0.381701,0.471609,0.491723,0.573877,0.626703,0.604795,0.677284,0.671094,0.655858,0.656113,0.707940,0.729135,0.729285,0.727657,0.701234
Rapporto di indebitamento\nAnno - 1,0.050769,-0.087772,0.037425,0.046400,0.110821,0.114969,0.078371,0.062516,0.095278,0.184793,0.213116,0.195260,0.285188,0.387817,0.346813
Rapporto di indebitamento\nAnno - 2,0.656220,0.488315,0.899603,0.840146,0.585793,0.584697,0.652339,0.773056,0.802018,0.795571,0.816280,0.796379,0.787703,0.653814,0.664457


In [155]:
df_evaluation.iloc[-15:-3]

,k = 1,k = 2,k = 3,k = 4,k = 5,k = 6,k = 7,k = 8,k = 9,k = 10,k = 11,k = 12,k = 13,k = 14,k = 15
Redditività del capitale proprio (ROE) (%)\n%\nAnno - 1,-3.768420,-2.041071,-0.704961,-0.405943,-0.299537,-0.193124,0.110489,0.218416,-0.045780,-0.659227,-0.429135,-0.343730,-0.176455,-0.279777,-0.361270
Redditività del capitale proprio (ROE) (%)\n%\nAnno - 2,3.274483,-12.428510,-9.169200,-5.722040,-4.010591,-2.599115,-2.254440,-4.811427,-3.904341,-3.848937,-3.637599,-3.015935,-2.545560,-1.575191,-1.377303
Redditività del capitale proprio (ROE) (%)\n%\nAnno - 3,75.474175,35.341689,29.078387,26.049291,25.992145,21.494050,20.068040,18.960241,18.278920,19.973869,18.407411,9.799941,10.157164,7.940219,7.565341
Redditività di tutto il capitale investito (ROI) (%)\n%\nAnno - 1,-0.005223,0.346067,0.402513,0.163067,0.233202,0.064433,0.016113,0.081675,0.116836,0.208625,0.215132,0.231986,0.162338,0.199690,0.110442
Redditività di tutto il capitale investito (ROI) (%)\n%\nAnno - 2,-0.141457,0.035211,-0.048249,-0.119396,-0.223148,-0.233776,-0.225636,-0.202021,-0.141664,-0.129094,-0.175125,-0.178914,-0.177564,-0.173054,-0.197470
Redditività di tutto il capitale investito (ROI) (%)\n%\nAnno - 3,0.069393,0.076258,-0.148081,-0.068862,-0.023943,-0.072960,0.094969,0.144500,0.054754,0.076485,0.019316,0.066874,0.062750,0.118923,0.101813
Rotaz. cap. investito (volte)\nAnno - 1,0.074574,0.089868,0.081830,0.083048,0.105009,0.116086,0.117946,0.108380,0.113073,0.109535,0.105587,0.102420,0.103958,0.107878,0.108080
Rotaz. cap. investito (volte)\nAnno - 2,0.131762,0.196193,0.166631,0.165088,0.150603,0.162261,0.164194,0.151868,0.159011,0.155918,0.159100,0.158504,0.161706,0.158881,0.162080
Rotaz. cap. investito (volte)\nAnno - 3,0.022618,0.044055,0.052468,0.046150,0.060498,0.067007,0.068510,0.074275,0.069771,0.060630,0.059905,0.060123,0.057678,0.062059,0.066045
Indice corrente\nAnno - 1,0.250231,0.349226,0.392338,0.357025,0.354531,0.362998,0.345296,0.325126,0.346964,0.346466,0.382970,0.437546,0.416073,0.420745,0.413838


In [156]:
df_evaluation.to_csv("results.csv")

In [157]:
fig = px.bar(df_evaluation.iloc[-15:-3].transpose(), barmode="group",title="Percentage mean errors")
fig.show()
fig.write_html("Percentage_mean_errors.html")

In [158]:
df_errors

,k = 1,k = 2,k = 3,k = 4,k = 5,k = 6,k = 7,k = 8,k = 9,k = 10,k = 11,k = 12,k = 13,k = 14,k = 15
MSE,160378.416916,155026.945165,163658.790773,245606.091246,324940.739416,491435.656825,692037.985366,596214.926151,548814.666080,498597.318855,550270.546815,506409.861051,450771.162486,342789.168570,229297.316997
MSE principal nan columns,182.170035,40.840061,21.804503,14.035731,14.532440,9.149939,8.202294,8.609428,7.093426,8.759611,7.777944,1.709760,1.813783,0.988433,1.082667


In [159]:
df_errors.to_csv("mse_.csv")

In [160]:
fig = px.bar(df_errors.transpose(),log_y=True, barmode="group",title="Mean squared errors (log scale)")
fig.show()
fig.write_html("Mean_squared_errors_log_scale.html")

# Clear trend! Greater k less the imputer's errors. Repeat the test n times for more robustness
# This method could be applied on more k values, and can be extended on other imputers